# SQL query from table names

In This notebook we are going to test if using just the name of the table, and a shord definition of its contect we can use a model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

In [77]:
!pip install dotenv
!pip install jupyter_bokeh

In [91]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('.env.txt'))

OPENAI_API_KEY  = ""

In [92]:
#Functio to call the model.
def return_OAI(user_message):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)
    context = []
    context.append({'role':'system', "content": user_message})

    response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=context,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [93]:
#Definition of the tables.
import pandas as pd

# Table and definitions sample
data = {
    'table': ['employees', 'salary', 'studies'],
    'definition': [
        "Employee information, name, position, and contact details.",
        "Salary details for each year, including bonuses and deductions.",
        "Educational studies, name of the institution, major, and graduation date."
    ]
}

df = pd.DataFrame(data)
print(df)

       table                                         definition
0  employees  Employee information, name, position, and cont...
1     salary  Salary details for each year, including bonuse...
2    studies  Educational studies, name of the institution, ...


In [94]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

In [95]:
print(text_tables)

employees: Employee information, name, position, and contact details.
salary: Salary details for each year, including bonuses and deductions.
studies: Educational studies, name of the institution, major, and graduation date.


In [96]:
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Questyion:
{question}
"""


In [97]:
#Creating the prompt, with the user questions and the tables definitions.
pqt1 = prompt_question_tables.format(tables=text_tables, question="Which employee has the highest salary?")

In [98]:
print(return_OAI(pqt1))

```json
["employees", "salary"]
```


In [99]:
pqt3 = prompt_question_tables.format(tables=text_tables,
                                     question="Which employee received the highest salary last year, and how much did they earn?")

In [100]:
print(return_OAI(pqt3))

```json
["employees", "salary"]
```


# Exercise
 - Complete the prompts similar to what we did in class.
     - Try a few versions if you have time
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [112]:
data = {
    'table': [
        'employees',
        'salary',
        'studies',
        'departments',
        'projects',
        'performance_reviews',
        'car'
    ],
    'definition': [
        "Employee information, name, position, and contact details.",
        "Salary details for each year, including bonuses and deductions.",
        "Educational studies, name of the institution, major, and graduation date.",
        "Department details, including department name and manager.",
        "Project details, including project name, start date, end date, and budget.",
        "Performance reviews with ratings, review dates, and feedback.",
        "Car for dayly use of each employer."]
}
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Questyion:
{question}
"""


In [105]:
pqt4 = prompt_question_tables.format(tables=text_tables, question="Which projects are the better payed employers involved with?")
print(return_OAI(pqt4))

```json
["employees", "salary"]
```


In [107]:
pqt4 = prompt_question_tables.format(tables=text_tables, question="Which emplyers get bonus per depatment per position and education?")
print(return_OAI(pqt4))

```json
["employees", "salary", "studies"]
```


In [106]:
pqt4 = prompt_question_tables.format(tables=text_tables, question="Which position and education are the employers that perform the better?")
print(return_OAI(pqt4))

```json
["employees", "studies", "salary"]
```


In [115]:
pqt4 = prompt_question_tables.format(tables=text_tables, question="Which employee has the highest annual salary, and what are their department details, educational background, current project details (including budgets), and average performance review rating?")
print(return_OAI(pqt4))

```json
{
  "tables": [
    "employees",
    "salary",
    "studies"
  ]
}
```


should use all the tables
